<a href="https://colab.research.google.com/github/aso1801031/AI_teach2020/blob/master/20201009AI0208_selectAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## sklearnでエラーが出るので、前のバージョンを入れ直す

In [8]:
# 現在のscikitーlearnをcolabのVMからアンインストール
!pip uninstall scikit-learn
# 前のバージョンを指定してvmにいんすとーる
!pip install scikit-learn==0.20 --ignore-installed

Uninstalling scikit-learn-0.20.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scikit_learn-0.20.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.20.0
  Using cached https://files.pythonhosted.org/packages/0c/b2/05be9b6da9ae4a4c54f537be22e95833f722742a02b1e355fdc09363877c/scikit_learn-0.20.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/14/d0475ceeaa047f3eae2dc7597c3d40228e1d5beee42c1730f3c96ffb5526/scipy-1.5.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/63/97/af8a92864a04bfa48f1b5c9b1f8bf2ccb2847f24530026f26dd223de4ca0/numpy-1.19.2-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.3 which is incompatible.
ERROR: datascience 0.10.6

### all_estimators()でループして各種のアルゴリズムで学習させ、比較する

In [9]:
# ライブラリ読み込み
import pandas as pd
# 学習用データとテスト検証用データを分割する
from sklearn.model_selection import train_test_split
# 正解率を算出するライブラリ
from sklearn.metrics import accuracy_score
# all_estimaios(推定機の全てを利用できるようにする)
from sklearn.utils.testing import all_estimators
# 警告クラス
import warnings

### Webからあやめのデータを読み込む

In [10]:
# urリンクから読み込むライブラリ
import urllib.request as req
# url用の文字列変数を定義
url ="https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/io/data/csv/iris.csv"
#リンクから保存するためのファイル名を定義
savefile = "iris.csv"
# requestクラスを使ってurlからデータをsavefileへコピーして保存
req.urlretrieve(url,savefile)
#pandasのreadメソッドを使ってcsvファイルを読み込む
iris_data = pd.read_csv(savefile,encoding="utf-8")
iris_data.head()


,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### アヤメデータを目的変数yと説明変数xに分離する

In [11]:
y = iris_data.loc[:, "Name"] #目的変数
x = iris_data.loc[:,["SepalLength","SepalWidth","PetalLength","PetalWidth"]] #説明変数

print(y.head())
print(x.head())

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Name, dtype: object
   SepalLength  SepalWidth  PetalLength  PetalWidth
0          5.1         3.5          1.4         0.2
1          4.9         3.0          1.4         0.2
2          4.7         3.2          1.3         0.2
3          4.6         3.1          1.5         0.2
4          5.0         3.6          1.4         0.2


### アヤメデータをさらに学習用とテストように分離する

In [12]:
# 学習用データ（train）と検証用データ（test）に分離
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,train_size=0.8,shuffle=True)
print(x_train.head())
print(y_train.head())
print(x_test.head())
print(y_test.head())

    SepalLength  SepalWidth  PetalLength  PetalWidth
74          6.4         2.9          4.3         1.3
49          5.0         3.3          1.4         0.2
20          5.4         3.4          1.7         0.2
41          4.5         2.3          1.3         0.3
5           5.4         3.9          1.7         0.4
74    Iris-versicolor
49        Iris-setosa
20        Iris-setosa
41        Iris-setosa
5         Iris-setosa
Name: Name, dtype: object
     SepalLength  SepalWidth  PetalLength  PetalWidth
147          6.5         3.0          5.2         2.0
1            4.9         3.0          1.4         0.2
81           5.5         2.4          3.7         1.0
98           5.1         2.5          3.0         1.1
19           5.1         3.8          1.5         0.3
147     Iris-virginica
1          Iris-setosa
81     Iris-versicolor
98     Iris-versicolor
19         Iris-setosa
Name: Name, dtype: object


### allAlgorithmsにclassifierを指定して「分類」でフィルターしたアルゴリズムの推定器群を取得する

In [13]:
# 推定器のリストをフィルター指定して取得
allAlgorithms = all_estimators(type_filter="classifier")
# ワーニングを無視する
warnings.filterwarnings("ignore")

### 推定器のリストをループしながら学習&推論させる

In [14]:
#推定器のリストをループ
for(name,algorithm) in allAlgorithms:
  #各アルゴリズムのオブジェクトを生成
  clf = algorithm() # 各アルゴリズムクラスのコンストラクトを実行
  #　学習させる
  clf.fit(x_train,y_train)
  # 推論する
  y_pred = clf.predict(x_test)
  # accuracy_scoreで正解率を算出して表示
  print(name,"正解率 = ",accuracy_score(y_test,y_pred) )

AdaBoostClassifier 正解率 =  1.0
BaggingClassifier 正解率 =  0.9666666666666667
BernoulliNB 正解率 =  0.23333333333333334
CalibratedClassifierCV 正解率 =  1.0
ComplementNB 正解率 =  0.5666666666666667
DecisionTreeClassifier 正解率 =  1.0
ExtraTreeClassifier 正解率 =  0.9333333333333333
ExtraTreesClassifier 正解率 =  0.9666666666666667
GaussianNB 正解率 =  1.0
GaussianProcessClassifier 正解率 =  1.0
GradientBoostingClassifier 正解率 =  1.0
KNeighborsClassifier 正解率 =  0.9666666666666667
LabelPropagation 正解率 =  0.9666666666666667
LabelSpreading 正解率 =  0.9666666666666667
LinearDiscriminantAnalysis 正解率 =  1.0
LinearSVC 正解率 =  1.0
LogisticRegression 正解率 =  0.9666666666666667
LogisticRegressionCV 正解率 =  0.9666666666666667
MLPClassifier 正解率 =  0.9666666666666667
MultinomialNB 正解率 =  0.6666666666666666
NearestCentroid 正解率 =  0.9666666666666667
NuSVC 正解率 =  1.0
PassiveAggressiveClassifier 正解率 =  0.5666666666666667
Perceptron 正解率 =  0.5666666666666667
QuadraticDiscriminantAnalysis 正解率 =  1.0
RadiusNeighborsClassifier 正解率 =  1.0


## クロスバリデーション 

### クロスバリデーション 関連のライブラリのインポート

In [15]:
# ライブラリのインポート
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

### クロスバリデーション を実行する

In [22]:
# K分割交差検証用のインスタンスを生成する
Kfold = KFold(n_splits=5,shuffle=True)
# 　すでに生成してあるall_astimatorsのインスタンスをループさせて、クロスバリデーション を行う
for(name,algorithm) in allAlgorithms:
  try:
    # 各アルゴリズムのインスタンスを取得
    clf = algorithm()
    #scoreの属性がある学習モデルだけに送る
    if hasattr(clf,"score"):
      scores = cross_val_score(clf,x,y,cv=kfold_cv)
      #k=n splits=5回分のクロスバリデーション のスコアを表示
      print(name,"の正解率")
      print(scores)
      #ifブロック終了
    #tryブロック終了
  except: #例外処理のブロック
    #何もしない
    pass
